# Introducción
En este notebook tenemos el código para hacer un gráfico y distintos indicadores para realizar un análisis técnico

In [1]:
# Librerias
import numpy as np
import pandas as pd

# Stock
import yfinance as yf
from lightweight_charts import Chart

In [2]:
# Descargamos los datos de apple mismo
start_date = "2024-01-01"
end_date = "2024-12-12"
interval='1d'

In [2]:


df = yf.download("AAPL", start=start_date, end=end_date)

[*********************100%***********************]  1 of 1 completed


In [3]:
# Limpiamos el multiindex
rename_columns = {'Open' : 'open', 
                  'High':  'high', 
                  'Low' : 'low', 
                  'Close' : 'close', 
                  'Adj Close' : 'adj close', 
                  'Volume' : 'volume'}
df.rename(columns=rename_columns, inplace=True)
df.columns = df.columns.droplevel('Ticker')

In [3]:
def get_bar_data(symbol, start_date, end_date, interval):
    df = yf.download(symbol, start=start_date, end=end_date, interval=interval)
    # Limpiamos el multiindex
    rename_columns = {'Open' : 'open', 
                    'High':  'high', 
                    'Low' : 'low', 
                    'Close' : 'close', 
                    'Adj Close' : 'adj close', 
                    'Volume' : 'volume'}
    df.rename(columns=rename_columns, inplace=True)
    df.columns = df.columns.droplevel('Ticker')
    return df

def on_search(chart, searched_string):  # Called when the user searches.
    new_data = get_bar_data(searched_string, chart.topbar['timeframe'].value)
    if new_data.empty:
        return
    chart.topbar['symbol'].set(searched_string)
    chart.set(new_data)


def on_timeframe_selection(chart):  # Called when the user changes the timeframe.
    new_data = get_bar_data(chart.topbar['symbol'].value, chart.topbar['timeframe'].value)
    if new_data.empty:
        return
    chart.set(new_data, True)

In [4]:
# Instanciamos el gráfico
chart = Chart(toolbox=True)

chart.events.search += on_search
chart.topbar.textbox('symbol', 'TSLA')

chart.topbar.switcher(
    'timeframe', ('1min', '5min', '1d'),
    func=on_timeframe_selection
)
df = get_bar_data('TSLA', start_date, end_date, interval)
chart.set(df)

# chart.horizontal_line(200, func=on_horizontal_line_move)

chart.show(block=True)

[*********************100%***********************]  1 of 1 completed
2025-01-22 14:45:03.376 python[4846:176450] +[IMKClient subclass]: chose IMKClient_Modern


RuntimeError: asyncio.run() cannot be called from a running event loop

2025-01-22 14:45:03.440 python[4846:176450] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-01-22 14:46:02.630 python[4846:176450] <class 'AttributeError'>: 'NSKVONotifying_NSApplication' object has no attribute 'abortModal_'
